<h1><center>Obtaining Stocks and Market Data + Fama-MacBeth Regression</center></h1>

## Course: Introduction to Programming 2477-2223 S1, Nova SBE


### Team:
*  ### Daniel Brosko (55036)
*  ### Joao Victor Rosa Omori (53135)
*  ### Marion Chevalier Montebrun (55246)
*  ### Parsa Rahimi Fetrati (54283)

## 1 Introduction

Portfolio analysis is a commonly used tool for asset pricing. It is useful for illustrating detailed information on the cross-sectional relationship between variables, and it also helps to uncover the relationship between those variables and future stock returns (Bali, Engle, and Murray, 2016). The biggest advantage for portfolio analysis is that it does not require any assumptions as it is a nonparametric approach. Because of its nonparametric characteristics, it could detect nonlinear relationship between variables as well (Bali, Engle, and Murray, 2016).

This paper will focus on Fama-MacBeth regression. The Fama-MacBeth regression analysis is to investigate the features of variable coefficients in the cross-sectional regression on return and targeted variables. Bai and Zhou (2015) applied the Fama-MacBeth regression on samples with different sizes and time ranges. They discovered the significant impact of sample size on stock returns and risk premium estimates.

The exact variables we examine the effect of are market return, short time reversal, and size of the firm (logarithm of market capital). We obtained the Market Cap data by scraping the macrotrends.net website, stocks data from yahoo finance (we found working source despite the official one being degraded), and the market data from professor Kenneth French's website.

For the analysis, we use 20 stocks from S&P 500 in this project. Sections 3 is focused on obtaining the data. Intermediate computations are covered in Section 4. In Section 5, we finally perform the Fama-MacBeth regression, and comment on findings of the model results.

The period on which we were doing our analyses covers the period from January 1, 2010 till October 31, 2022. We started working with daily prices and returns for beta estimates. Then we switched to the monthly closing data for further analyses covered here. The market data (F-F factors and Market Cap), were all downloaded in daily format and afterwards transformed into monthly data. Additionals comments are present across the particular "chunks" of the code, and the particular variables are properly defined in the "Empirical Methodology".

### Empirical methodology

The parameters in FM regression are estimated in two steps:

* First regress each of n asset returns against m proposed risk factors to determine each asset's beta exposures.

\begin{array}{lcr}R_{1,t}=\alpha _{1}+\beta _{1,F_{1}}F_{1,t}+\beta _{1,F_{2}}F_{2,t}+\cdots +\beta _{1,F_{m}}F_{m,t}+\epsilon _{1,t}\\R_{2,t}=\alpha _{2}+\beta _{2,F_{1}}F_{1,t}+\beta _{2,F_{2}}F_{2,t}+\cdots +\beta _{2,F_{m}}F_{m,t}+\epsilon _{2,t}\\\vdots \\R_{n,t}=\alpha _{n}+\beta _{n,F_{1}}F_{1,t}+\beta _{n,F_{2}}F_{2,t}+\cdots +\beta _{n,F_{m}}F_{m,t}+\epsilon _{n,t}\end{array}

* Then regress all asset returns for each of T time periods against the previously estimated betas to determine the risk premium for each factor.

\begin{array}{lcr}R_{i,1}=\gamma _{1,0}+\gamma _{1,1}{\hat {\beta }}_{i,F_{1}}+\gamma _{1,2}{\hat {\beta }}_{i,F_{2}}+\cdots +\gamma _{1,m}{\hat {\beta }}_{i,F_{m}}+\epsilon _{i,1}\\R_{i,2}=\gamma _{2,0}+\gamma _{2,1}{\hat {\beta }}_{i,F_{1}}+\gamma _{2,2}{\hat {\beta }}_{i,F_{2}}+\cdots +\gamma _{2,m}{\hat {\beta }}_{i,F_{m}}+\epsilon _{i,2}\\\vdots \\R_{i,T}=\gamma _{T,0}+\gamma _{T,1}{\hat {\beta }}_{i,F_{1}}+\gamma _{T,2}{\hat {\beta }}_{i,F_{2}}+\cdots +\gamma _{T,m}{\hat {\beta }}_{i,F_{m}}+\epsilon _{i,T}\end{array} (source: https://en.wikipedia.org/wiki/Fama–MacBeth_regression.)


From coding point of view, the main part is this notebook, while we also use .py scripts written from the scratch.
The project is also available at GitHub: https://github.com/tigerbroko/Nova-SBE-Introduction-to-Programming.git (SSH: git@github.com:tigerbroko/Nova-SBE-Introduction-to-Programming.git).

## 2 General packages loading

In [1]:
import pandas as pd
import numpy as np
import re
import requests
from bs4 import BeautifulSoup
import datetime
import time
import yfinance as yf
from sklearn.linear_model import LinearRegression
import statistics

#importing own functions
import scraping_functions as sf
import computing_functions as cf
import stock_prices as sp

In [2]:
# pip list # used for requirements.txt to get the version numbers

In [3]:
# loading tickers of selected stocks
tickers = sf.load_tickers("tickers_set.csv")

## 3 Obtaining the Data
### 3.1 Market Capitalization (Size) Data Scraping

Here we scrape the market capitalizations of the firms using own scraping function from scraping_function.py.
The data are in billions of USD.

In [4]:
starting_time = time.time()

# we perform the scraping of market capitalization for all desired stocks 
mkt_cap_df_list = []

for ticker in tickers:
    mkt_cap_df_list.append(sf.mkt_cap_scraper(ticker))


elapsed_time = time.time() - starting_time
print('Elapsed time:', round(elapsed_time,3) , 'seconds.')

# mkt_cap_df_list

Elapsed time: 1.498 seconds.


In [ ]:
# merging mkt. capital for all stocks, and renaming the columns according to particular stocks
df_mkt_cap = mkt_cap_df_list[0]

for dataframe in mkt_cap_df_list[1:] :
    df_mkt_cap = df_mkt_cap.merge(dataframe, on='Date', how = 'inner')

mc_columns_names = ['Date']
mc_columns_names.extend(tickers)
df_mkt_cap.columns = mc_columns_names
df_mkt_cap.set_index('Date', inplace=True)

# using size instead of mkt_cap to rule out skewness
df_size = np.log(df_mkt_cap + 0.000001)

### 3.2 Obtaining Stock Prices

In [6]:
# downloading stocks prices - designed to save as csv, so we have "back-up" if something goes wrong later
success_tickers, failed_tickers = sp.get_stocks(tickers)

TER - SUCCESS
ADM - SUCCESS
TMUS - SUCCESS
IDXX - SUCCESS
LNC - SUCCESS
WST - SUCCESS
AMZN - SUCCESS
DOV - SUCCESS
ORLY - SUCCESS
BSX - SUCCESS
USB - SUCCESS
ETR - SUCCESS
AZO - SUCCESS
PVH - SUCCESS
BIIB - SUCCESS
ETN - SUCCESS
PFE - SUCCESS
WMB - SUCCESS
FITB - SUCCESS
CHRW - SUCCESS
Elapsed time: 21.827 seconds.


In [7]:
# loading stock prices into list of dataframes using own function
stocks_df_list = sp.load_stocks(success_tickers)
# stocks_df_list

In [ ]:
df_close_prices = stocks_df_list[0][['Date','Adj Close']]
 
for index in range(1, len(stocks_df_list)):
    dataframe = stocks_df_list[index][['Date','Adj Close']]
    df_close_prices = df_close_prices.merge(dataframe, on='Date', how = 'inner')

cp_columns_names = ['Date']
cp_columns_names.extend(success_tickers)
df_close_prices.columns = cp_columns_names
df_close_prices['Date'] = pd.to_datetime(df_close_prices['Date']).dt.date
df_close_prices.set_index('Date', inplace=True)

In [9]:
# Making returns from daily prices
# daily_returns = df_close_prices.pct_change(1) # 1 for ONE DAY lookback
daily_returns = np.log(df_close_prices/df_close_prices.shift())
daily_returns.dropna(inplace=True)
daily_returns.head()

# monthly_return = prices.pct_change(21) # 21 for ONE MONTH lookback
# annual_return = prices.pct_change(252)

,TER,ADM,TMUS,IDXX,LNC,WST,AMZN,DOV,ORLY,BSX,USB,ETR,AZO,PVH,BIIB,ETN,PFE,WMB,FITB,CHRW
Date,,,,,,,,,,,,,,,,,,,,
2009-12-09,0.011288,-0.010768,-0.026550,-0.005999,0.007538,-0.002321,-0.021099,-0.005363,-0.008706,0.000000,-0.003487,-0.000494,-0.003767,-0.018150,0.002286,0.000000,0.026668,0.000517,0.009766,0.000871
2009-12-10,-0.015425,0.000984,0.030908,0.009658,0.005286,0.006947,0.030525,0.001954,-0.003451,0.027909,0.005661,0.010699,-0.005839,0.007098,0.001867,0.005139,0.004377,0.013864,-0.020619,0.003995
2009-12-11,-0.003113,-0.000656,0.007220,0.002688,-0.014160,0.013245,-0.009127,0.001950,-0.001064,-0.005750,0.000868,0.015897,0.002442,0.022231,0.008049,0.001242,-0.001093,-0.001020,0.009872,0.007082
2009-12-14,0.069270,-0.008564,-0.011578,0.006878,0.012841,0.009820,-0.020865,0.012822,0.009273,0.029548,-0.000434,0.008392,0.017688,0.009099,0.004512,0.019355,0.005449,0.036586,0.014627,0.016050
2009-12-15,0.002905,0.013472,0.067536,0.008343,-0.027204,0.001503,-0.008792,-0.005786,-0.010604,-0.007870,-0.028617,-0.009115,-0.010461,-0.012068,0.011394,-0.001523,-0.005996,0.013200,-0.052670,0.011284


### 3.3 Market Return from prof. Kenneth French's website 

We dowloaded data about the market from https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/data_library.html , however, we need to change the date into suitable format.

In [10]:
data_market = pd.read_csv("F-F_Research_Data_Factors_daily.CSV", skiprows=3)
data_market = data_market[:-1]

data_market.rename(columns = {'Unnamed: 0': 'Date'}, inplace = True)

# using simple lambda function to put dates into correct format
date_format_lambda = lambda x: datetime.datetime.strptime(x, '%Y%m%d')
data_market['Date'] = data_market['Date'].map(date_format_lambda)
data_market['Date'] = pd.to_datetime(data_market['Date']).dt.date
data_market.set_index('Date', inplace=True)
data_market.head()

,Mkt-RF,SMB,HML,RF
Date,,,,
1926-07-01,0.10,-0.25,-0.27,0.009
1926-07-02,0.45,-0.33,-0.06,0.009
1926-07-06,0.17,0.30,-0.39,0.009
1926-07-07,0.09,-0.58,0.02,0.009
1926-07-08,0.21,-0.38,0.19,0.009


### 3.4 Short-Time-Reversal from prof. Kenneth French's website 
We dowloaded STR data from https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/data_library.html
and proceed in the similar fashion to subsection 3.3.

In [11]:
st_reversal = pd.read_csv("F-F_ST_Reversal_Factor_daily.CSV", skiprows=12)
st_reversal = st_reversal[:-1]
st_reversal.rename(columns = {'Unnamed: 0': 'Date'}, inplace = True)

# using simple lambda function to put dates into correct format
date_format_lambda = lambda x: datetime.datetime.strptime(x, '%Y%m%d')
st_reversal['Date'] = st_reversal['Date'].map(date_format_lambda)
st_reversal['Date'] = pd.to_datetime(st_reversal['Date']).dt.date
st_reversal.set_index('Date', inplace=True)
st_reversal.head()

,ST_Rev
Date,
1926-01-26,0.14
1926-01-27,0.62
1926-01-28,0.22
1926-01-29,0.58
1926-01-30,0.46


### 3.4 Filtering data for reasonable time interval

In [12]:
size = df_size.query("Date >= datetime.date(2010,1,1) and Date <= datetime.date(2022,10,31)")
returns = daily_returns.query("Date >= datetime.date(2010,1,1) and Date <= datetime.date(2022,10,31)")
market = data_market.query("Date >= datetime.date(2010,1,1) and Date <= datetime.date(2022,10,31)")
reversal = st_reversal.query("Date >= datetime.date(2010,1,1) and Date <= datetime.date(2022,10,31)")

Here, we adjust stock returns for risk-free rate, to simplify the linear regression (we get rid of the intercept, which is rf rate).

In [13]:
# adjusting stock returns to risk-free rate
risk_free_rate = market.loc[:, 'RF']
risk_free_rate = pd.DataFrame(risk_free_rate)
risk_free_rate_column = risk_free_rate.copy()

for column in range(1, len(success_tickers)):
    risk_free_rate = pd.concat([risk_free_rate, risk_free_rate_column], axis=1)

excess_returns = returns.subtract(risk_free_rate.values)

excess_returns

,TER,ADM,TMUS,IDXX,LNC,WST,AMZN,DOV,ORLY,BSX,USB,ETR,AZO,PVH,BIIB,ETN,PFE,WMB,FITB,CHRW
Date,,,,,,,,,,,,,,,,,,,,
2010-01-04,0.021209,0.005097,0.043597,0.011718,0.039406,0.006864,-0.004620,0.017864,0.009919,0.001111,0.016304,0.003781,-0.000253,0.010272,0.002613,0.010943,0.039876,0.024832,0.037246,0.010333
2010-01-05,0.003643,0.005387,0.002506,0.001663,0.020651,-0.012749,0.005883,-0.004733,0.001557,0.003324,0.025033,-0.016075,-0.008388,0.036784,-0.004859,-0.005925,-0.014366,0.012417,0.026331,-0.020946
2010-01-06,-0.009132,-0.002532,-0.069977,0.002397,0.015026,-0.009799,-0.018282,0.011790,-0.004419,0.013187,0.010600,0.005059,-0.009425,0.023640,0.000936,-0.004389,-0.003221,0.031934,0.005758,-0.013339
2010-01-07,0.005489,-0.010511,0.009352,0.037235,0.023591,0.003105,-0.017160,0.021796,-0.003131,-0.007671,0.019215,-0.012010,0.013182,0.000000,-0.008269,0.014658,-0.003770,-0.007108,0.058539,-0.009638
2010-01-08,0.010890,-0.012567,-0.057471,-0.008016,-0.015419,-0.000258,0.026717,0.028266,-0.008924,-0.009950,0.001654,-0.009638,-0.012925,-0.001605,0.020361,0.022198,0.008062,0.007993,0.000902,0.013294
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-10-25,0.016880,0.009619,-0.001641,0.010255,0.013569,0.040840,-0.004511,-0.002004,-0.001584,-0.003222,0.010170,0.012276,0.002356,0.047404,-0.014867,0.020155,-0.009903,0.012830,0.017114,-0.007195
2022-10-26,0.021121,0.015527,-0.009508,0.005746,-0.027826,0.009612,-0.052824,-0.008881,-0.008772,0.000795,-0.001995,-0.011286,-0.017666,-0.004292,0.013803,-0.014635,-0.000743,-0.001625,-0.006484,0.006774
2022-10-27,-0.040488,-0.005247,-0.012492,-0.017047,-0.000641,-0.150705,-0.052485,-0.007322,0.033969,-0.016038,-0.018340,-0.008044,0.028674,-0.027370,-0.020171,-0.002992,-0.017972,-0.000172,-0.015516,-0.007896


Just to remind what objects (dataframes) are important fot us now, we obtained the following dataframes that we are going to use:

* size - logarithm of market capitalization    
* excess_returns - daily stock returns adjusted to risk-free rate
* market - FF market factors
* reversal - Short-Time-Reversal market factor

## 4 Intermediate Computations
### 4.1 Estimating Market Betas

We estimate stocks's sensitivity to the market (beta factors) on daily data using linear regression (implemented in the market_betas function from computation_functions.py script).

In [14]:
mkt_betas = cf.market_betas(excess_returns, market)
mkt_betas.head()

Elapsed time: 11.31 seconds.


,TER,ADM,TMUS,IDXX,LNC,WST,AMZN,DOV,ORLY,BSX,USB,ETR,AZO,PVH,BIIB,ETN,PFE,WMB,FITB,CHRW
Date,,,,,,,,,,,,,,,,,,,,
2011-01-04,0.017935,0.007381,0.011974,0.009055,0.018687,0.009052,0.010824,0.013625,0.007077,0.009581,0.011858,0.006959,0.004700,0.015762,0.007382,0.012936,0.007492,0.014149,0.017356,0.008058
2011-01-05,0.017977,0.007416,0.011807,0.009080,0.018637,0.009105,0.010932,0.013653,0.007119,0.009666,0.011879,0.006986,0.004768,0.015861,0.007439,0.012991,0.007343,0.014147,0.017316,0.008082
2011-01-06,0.017931,0.007414,0.011838,0.009078,0.018613,0.009118,0.010941,0.013630,0.007117,0.009667,0.011852,0.006981,0.004751,0.015846,0.007445,0.012999,0.007362,0.014128,0.017306,0.008083
2011-01-07,0.017933,0.007398,0.011895,0.009077,0.018613,0.009125,0.010952,0.013630,0.007133,0.009669,0.011856,0.006982,0.004765,0.015847,0.007441,0.013006,0.007361,0.014116,0.017313,0.008089
2011-01-10,0.017924,0.007408,0.011905,0.009041,0.018591,0.009127,0.010977,0.013611,0.007139,0.009693,0.011842,0.006996,0.004748,0.015876,0.007452,0.012994,0.007363,0.014120,0.017248,0.008104


### 4.2 Estimating Size Betas

We estimate stocks's sensitivity to the size on daily data using linear regression (implemented in the size_betas function from computation_functions.py script).

In [15]:
size_betas = cf.size_betas(excess_returns, size)
size_betas.head()

stock:TER, row: 0
stock:TER, row: 1
stock:TER, row: 2
stock:TER, row: 3
stock:TER, row: 4
stock:TER, row: 5
stock:TER, row: 6
stock:TER, row: 7
stock:TER, row: 8
stock:TER, row: 9
stock:TER, row: 10
stock:TER, row: 11
stock:TER, row: 12
stock:TER, row: 13
stock:TER, row: 14
stock:TER, row: 15
stock:TER, row: 16
stock:TER, row: 17
stock:TER, row: 18
stock:TER, row: 19
stock:TER, row: 20
stock:TER, row: 21
stock:TER, row: 22
stock:TER, row: 23
stock:TER, row: 24
stock:TER, row: 25
stock:TER, row: 26
stock:TER, row: 27
stock:TER, row: 28
stock:TER, row: 29
stock:TER, row: 30
stock:TER, row: 31
stock:TER, row: 32
stock:TER, row: 33
stock:TER, row: 34
stock:TER, row: 35
stock:TER, row: 36
stock:TER, row: 37
stock:TER, row: 38
stock:TER, row: 39
stock:TER, row: 40
stock:TER, row: 41
stock:TER, row: 42
stock:TER, row: 43
stock:TER, row: 44
stock:TER, row: 45
stock:TER, row: 46
stock:TER, row: 47
stock:TER, row: 48
stock:TER, row: 49
stock:TER, row: 50
stock:TER, row: 51
stock:TER, row: 52
sto

stock:TER, row: 941
stock:TER, row: 942
stock:TER, row: 943
stock:TER, row: 944
stock:TER, row: 945
stock:TER, row: 946
stock:TER, row: 947
stock:TER, row: 948
stock:TER, row: 949
stock:TER, row: 950
stock:TER, row: 951
stock:TER, row: 952
stock:TER, row: 953
stock:TER, row: 954
stock:TER, row: 955
stock:TER, row: 956
stock:TER, row: 957
stock:TER, row: 958
stock:TER, row: 959
stock:TER, row: 960
stock:TER, row: 961
stock:TER, row: 962
stock:TER, row: 963
stock:TER, row: 964
stock:TER, row: 965
stock:TER, row: 966
stock:TER, row: 967
stock:TER, row: 968
stock:TER, row: 969
stock:TER, row: 970
stock:TER, row: 971
stock:TER, row: 972
stock:TER, row: 973
stock:TER, row: 974
stock:TER, row: 975
stock:TER, row: 976
stock:TER, row: 977
stock:TER, row: 978
stock:TER, row: 979
stock:TER, row: 980
stock:TER, row: 981
stock:TER, row: 982
stock:TER, row: 983
stock:TER, row: 984
stock:TER, row: 985
stock:TER, row: 986
stock:TER, row: 987
stock:TER, row: 988
stock:TER, row: 989
stock:TER, row: 990


stock:TER, row: 1587
stock:TER, row: 1588
stock:TER, row: 1589
stock:TER, row: 1590
stock:TER, row: 1591
stock:TER, row: 1592
stock:TER, row: 1593
stock:TER, row: 1594
stock:TER, row: 1595
stock:TER, row: 1596
stock:TER, row: 1597
stock:TER, row: 1598
stock:TER, row: 1599
stock:TER, row: 1600
stock:TER, row: 1601
stock:TER, row: 1602
stock:TER, row: 1603
stock:TER, row: 1604
stock:TER, row: 1605
stock:TER, row: 1606
stock:TER, row: 1607
stock:TER, row: 1608
stock:TER, row: 1609
stock:TER, row: 1610
stock:TER, row: 1611
stock:TER, row: 1612
stock:TER, row: 1613
stock:TER, row: 1614
stock:TER, row: 1615
stock:TER, row: 1616
stock:TER, row: 1617
stock:TER, row: 1618
stock:TER, row: 1619
stock:TER, row: 1620
stock:TER, row: 1621
stock:TER, row: 1622
stock:TER, row: 1623
stock:TER, row: 1624
stock:TER, row: 1625
stock:TER, row: 1626
stock:TER, row: 1627
stock:TER, row: 1628
stock:TER, row: 1629
stock:TER, row: 1630
stock:TER, row: 1631
stock:TER, row: 1632
stock:TER, row: 1633
stock:TER, ro

stock:TER, row: 2430
stock:TER, row: 2431
stock:TER, row: 2432
stock:TER, row: 2433
stock:TER, row: 2434
stock:TER, row: 2435
stock:TER, row: 2436
stock:TER, row: 2437
stock:TER, row: 2438
stock:TER, row: 2439
stock:TER, row: 2440
stock:TER, row: 2441
stock:TER, row: 2442
stock:TER, row: 2443
stock:TER, row: 2444
stock:TER, row: 2445
stock:TER, row: 2446
stock:TER, row: 2447
stock:TER, row: 2448
stock:TER, row: 2449
stock:TER, row: 2450
stock:TER, row: 2451
stock:TER, row: 2452
stock:TER, row: 2453
stock:TER, row: 2454
stock:TER, row: 2455
stock:TER, row: 2456
stock:TER, row: 2457
stock:TER, row: 2458
stock:TER, row: 2459
stock:TER, row: 2460
stock:TER, row: 2461
stock:TER, row: 2462
stock:TER, row: 2463
stock:TER, row: 2464
stock:TER, row: 2465
stock:TER, row: 2466
stock:TER, row: 2467
stock:TER, row: 2468
stock:TER, row: 2469
stock:TER, row: 2470
stock:TER, row: 2471
stock:TER, row: 2472
stock:TER, row: 2473
stock:TER, row: 2474
stock:TER, row: 2475
stock:TER, row: 2476
stock:TER, ro

stock:ADM, row: 133
stock:ADM, row: 134
stock:ADM, row: 135
stock:ADM, row: 136
stock:ADM, row: 137
stock:ADM, row: 138
stock:ADM, row: 139
stock:ADM, row: 140
stock:ADM, row: 141
stock:ADM, row: 142
stock:ADM, row: 143
stock:ADM, row: 144
stock:ADM, row: 145
stock:ADM, row: 146
stock:ADM, row: 147
stock:ADM, row: 148
stock:ADM, row: 149
stock:ADM, row: 150
stock:ADM, row: 151
stock:ADM, row: 152
stock:ADM, row: 153
stock:ADM, row: 154
stock:ADM, row: 155
stock:ADM, row: 156
stock:ADM, row: 157
stock:ADM, row: 158
stock:ADM, row: 159
stock:ADM, row: 160
stock:ADM, row: 161
stock:ADM, row: 162
stock:ADM, row: 163
stock:ADM, row: 164
stock:ADM, row: 165
stock:ADM, row: 166
stock:ADM, row: 167
stock:ADM, row: 168
stock:ADM, row: 169
stock:ADM, row: 170
stock:ADM, row: 171
stock:ADM, row: 172
stock:ADM, row: 173
stock:ADM, row: 174
stock:ADM, row: 175
stock:ADM, row: 176
stock:ADM, row: 177
stock:ADM, row: 178
stock:ADM, row: 179
stock:ADM, row: 180
stock:ADM, row: 181
stock:ADM, row: 182


...

stock:FITB, row: 1986
stock:FITB, row: 1987
stock:FITB, row: 1988
stock:FITB, row: 1989
stock:FITB, row: 1990
stock:FITB, row: 1991
stock:FITB, row: 1992
stock:FITB, row: 1993
stock:FITB, row: 1994
stock:FITB, row: 1995
stock:FITB, row: 1996
stock:FITB, row: 1997
stock:FITB, row: 1998
stock:FITB, row: 1999
stock:FITB, row: 2000
stock:FITB, row: 2001
stock:FITB, row: 2002
stock:FITB, row: 2003
stock:FITB, row: 2004
stock:FITB, row: 2005
stock:FITB, row: 2006
stock:FITB, row: 2007
stock:FITB, row: 2008
stock:FITB, row: 2009
stock:FITB, row: 2010
stock:FITB, row: 2011
stock:FITB, row: 2012
stock:FITB, row: 2013
stock:FITB, row: 2014
stock:FITB, row: 2015
stock:FITB, row: 2016
stock:FITB, row: 2017
stock:FITB, row: 2018
stock:FITB, row: 2019
stock:FITB, row: 2020
stock:FITB, row: 2021
stock:FITB, row: 2022
stock:FITB, row: 2023
stock:FITB, row: 2024
stock:FITB, row: 2025
stock:FITB, row: 2026
stock:FITB, row: 2027
stock:FITB, row: 2028
stock:FITB, row: 2029
stock:FITB, row: 2030
stock:FITB

stock:CHRW, row: 206
stock:CHRW, row: 207
stock:CHRW, row: 208
stock:CHRW, row: 209
stock:CHRW, row: 210
stock:CHRW, row: 211
stock:CHRW, row: 212
stock:CHRW, row: 213
stock:CHRW, row: 214
stock:CHRW, row: 215
stock:CHRW, row: 216
stock:CHRW, row: 217
stock:CHRW, row: 218
stock:CHRW, row: 219
stock:CHRW, row: 220
stock:CHRW, row: 221
stock:CHRW, row: 222
stock:CHRW, row: 223
stock:CHRW, row: 224
stock:CHRW, row: 225
stock:CHRW, row: 226
stock:CHRW, row: 227
stock:CHRW, row: 228
stock:CHRW, row: 229
stock:CHRW, row: 230
stock:CHRW, row: 231
stock:CHRW, row: 232
stock:CHRW, row: 233
stock:CHRW, row: 234
stock:CHRW, row: 235
stock:CHRW, row: 236
stock:CHRW, row: 237
stock:CHRW, row: 238
stock:CHRW, row: 239
stock:CHRW, row: 240
stock:CHRW, row: 241
stock:CHRW, row: 242
stock:CHRW, row: 243
stock:CHRW, row: 244
stock:CHRW, row: 245
stock:CHRW, row: 246
stock:CHRW, row: 247
stock:CHRW, row: 248
stock:CHRW, row: 249
stock:CHRW, row: 250
stock:CHRW, row: 251
stock:CHRW, row: 252
stock:CHRW, r

stock:CHRW, row: 1293
stock:CHRW, row: 1294
stock:CHRW, row: 1295
stock:CHRW, row: 1296
stock:CHRW, row: 1297
stock:CHRW, row: 1298
stock:CHRW, row: 1299
stock:CHRW, row: 1300
stock:CHRW, row: 1301
stock:CHRW, row: 1302
stock:CHRW, row: 1303
stock:CHRW, row: 1304
stock:CHRW, row: 1305
stock:CHRW, row: 1306
stock:CHRW, row: 1307
stock:CHRW, row: 1308
stock:CHRW, row: 1309
stock:CHRW, row: 1310
stock:CHRW, row: 1311
stock:CHRW, row: 1312
stock:CHRW, row: 1313
stock:CHRW, row: 1314
stock:CHRW, row: 1315
stock:CHRW, row: 1316
stock:CHRW, row: 1317
stock:CHRW, row: 1318
stock:CHRW, row: 1319
stock:CHRW, row: 1320
stock:CHRW, row: 1321
stock:CHRW, row: 1322
stock:CHRW, row: 1323
stock:CHRW, row: 1324
stock:CHRW, row: 1325
stock:CHRW, row: 1326
stock:CHRW, row: 1327
stock:CHRW, row: 1328
stock:CHRW, row: 1329
stock:CHRW, row: 1330
stock:CHRW, row: 1331
stock:CHRW, row: 1332
stock:CHRW, row: 1333
stock:CHRW, row: 1334
stock:CHRW, row: 1335
stock:CHRW, row: 1336
stock:CHRW, row: 1337
stock:CHRW

stock:CHRW, row: 2141
stock:CHRW, row: 2142
stock:CHRW, row: 2143
stock:CHRW, row: 2144
stock:CHRW, row: 2145
stock:CHRW, row: 2146
stock:CHRW, row: 2147
stock:CHRW, row: 2148
stock:CHRW, row: 2149
stock:CHRW, row: 2150
stock:CHRW, row: 2151
stock:CHRW, row: 2152
stock:CHRW, row: 2153
stock:CHRW, row: 2154
stock:CHRW, row: 2155
stock:CHRW, row: 2156
stock:CHRW, row: 2157
stock:CHRW, row: 2158
stock:CHRW, row: 2159
stock:CHRW, row: 2160
stock:CHRW, row: 2161
stock:CHRW, row: 2162
stock:CHRW, row: 2163
stock:CHRW, row: 2164
stock:CHRW, row: 2165
stock:CHRW, row: 2166
stock:CHRW, row: 2167
stock:CHRW, row: 2168
stock:CHRW, row: 2169
stock:CHRW, row: 2170
stock:CHRW, row: 2171
stock:CHRW, row: 2172
stock:CHRW, row: 2173
stock:CHRW, row: 2174
stock:CHRW, row: 2175
stock:CHRW, row: 2176
stock:CHRW, row: 2177
stock:CHRW, row: 2178
stock:CHRW, row: 2179
stock:CHRW, row: 2180
stock:CHRW, row: 2181
stock:CHRW, row: 2182
stock:CHRW, row: 2183
stock:CHRW, row: 2184
stock:CHRW, row: 2185
stock:CHRW

,TER,ADM,TMUS,IDXX,LNC,WST,AMZN,DOV,ORLY,BSX,USB,ETR,AZO,PVH,BIIB,ETN,PFE,WMB,FITB,CHRW
Date,,,,,,,,,,,,,,,,,,,,
2011-01-04,0.001348,0.000166,0.001597,0.000327,0.000036,0.000617,0.000139,0.000365,0.000581,-0.000579,0.000009,-0.000400,0.000603,0.000731,0.000076,0.000537,-0.000139,0.000029,0.000440,0.000300
2011-01-05,0.001227,0.000164,0.001757,0.000250,-0.000017,0.000121,0.000148,0.000324,0.000462,-0.000610,-0.000015,-0.000384,0.000531,0.000608,0.000054,0.000513,-0.000158,-0.000022,0.000376,0.000258
2011-01-06,0.001075,0.000164,0.001858,0.000256,-0.000047,0.000425,0.000156,0.000312,0.000465,-0.000605,-0.000038,-0.000371,0.000521,0.000593,0.000067,0.000533,-0.000141,-0.000039,0.000364,0.000279
2011-01-07,0.001143,0.000173,0.001728,0.000243,-0.000105,0.000432,0.000164,0.000271,0.000400,-0.000660,-0.000070,-0.000388,0.000494,0.000450,0.000080,0.000516,-0.000135,-0.000093,0.000311,0.000294
2011-01-10,0.001210,0.000177,0.001574,0.000107,-0.000148,0.000283,0.000178,0.000228,0.000405,-0.000677,-0.000097,-0.000364,0.000480,0.000319,0.000094,0.000495,-0.000125,-0.000056,0.000226,0.000308


### 4.3 Estimating STR Betas

We estimate stocks's sensitivity to the STR factor on daily data using linear regression (implemented in the str_betas function from computation_functions.py script).

In [16]:
str_betas = cf.str_betas(excess_returns, reversal)
str_betas.head()

Elapsed time: 11.516 seconds.


,TER,ADM,TMUS,IDXX,LNC,WST,AMZN,DOV,ORLY,BSX,USB,ETR,AZO,PVH,BIIB,ETN,PFE,WMB,FITB,CHRW
Date,,,,,,,,,,,,,,,,,,,,
2011-01-04,0.006474,0.003697,0.002748,0.006012,0.008347,0.004482,0.001955,0.005945,0.003569,0.003264,0.008570,0.003547,0.002324,0.005968,0.003877,0.008848,0.002873,0.005798,0.012959,0.003223
2011-01-05,0.006487,0.003700,0.002796,0.006016,0.008376,0.004481,0.001953,0.005955,0.003561,0.003260,0.008579,0.003555,0.002312,0.005966,0.003875,0.008854,0.002904,0.005811,0.012982,0.003226
2011-01-06,0.006628,0.003717,0.002669,0.006016,0.008514,0.004362,0.001920,0.005987,0.003560,0.003248,0.008767,0.003486,0.002327,0.006145,0.003816,0.008776,0.002756,0.005903,0.013103,0.003114
2011-01-07,0.006559,0.003533,0.002440,0.006058,0.008726,0.004329,0.001818,0.006149,0.003682,0.003436,0.008967,0.003566,0.002362,0.006464,0.003794,0.008825,0.002720,0.006183,0.013290,0.003030
2011-01-10,0.006578,0.003576,0.002356,0.005957,0.008675,0.004310,0.001852,0.006093,0.003688,0.003411,0.008905,0.003603,0.002343,0.006376,0.003816,0.008792,0.002749,0.006236,0.013168,0.003049


### 4.4 Transforming Data to Monthly Values

In [17]:
# firstly restricting to same time range, since to compute betas, we needed to give up of some data
size = size.iloc[253:, :]
excess_returns = excess_returns.iloc[253:, :]
market = market.iloc[253:, :]
reversal = reversal.iloc[253:, :]

In [18]:
# to monthly
monthly_size = sp.daily_to_monthly(size)
monthly_returns = sp.daily_to_monthly(excess_returns)
monthly_market = sp.daily_to_monthly(market)
monthly_reversal = sp.daily_to_monthly(reversal)
monthly_mkt_betas = sp.daily_to_monthly(mkt_betas)
monthly_size_betas = sp.daily_to_monthly(size_betas)
monthly_str_betas = sp.daily_to_monthly(str_betas)

## 5 Fama-MacBeth
### 5.1 Estimating Gamma Coefficients
Now we perform the second part of the regression - estimating the coefficients for excess returns on beta factors we estimated above in section 4.

In [19]:
gammas = cf.fama_macbeth(monthly_returns, monthly_mkt_betas, monthly_size_betas, monthly_str_betas)
gammas.head()

,intercept,mkt,size,str
Date,,,,
2011-01-31,-0.003352,0.092847,1.919403,0.333450
2011-02-28,-0.000650,0.265117,0.942599,-0.249414
2011-03-31,0.002725,-0.030486,1.386462,-0.368425
2011-04-30,0.005715,-0.060669,-1.655870,-0.463949
2011-05-31,0.000667,-0.178482,1.676451,-0.001218


We stored all the coefficients into the dataframe above (gammas), and finally do the statistics on obtained coefficients. The results of Fama-MacBeth regression are displayed below.

In [20]:
results = cf.results_table(gammas)
results

,intercept,mkt,size,str
statistics,,,,
mean,-0.001332,-0.023097,0.154813,-0.00407
st. dev,0.004145,0.368388,1.547875,0.351767
t-stat,-0.321387,-0.062698,0.100017,-0.011569


We estimated coefficients and t-stats for the case when we are controlling all sort variables (market, size, and STR). For the size - we added epsilon=0.000001 to the market cap and then took logarithm of it, to prevent the ill behaviour of the estimates, since the log(0)= - infinity.

From the Fama-Macbeth Regression, the values for intercept, market beta, size, and short-term reversal are -0.001332, -0.023097, 0.154813 and -0.00407, respectively. The regression model for excess returns is therefore constructed as follows: 

$$E(R) = -0.023097*mkt~beta + 0.154813* size -0.00407*str - 0.001332$$

From the results, it could appear that creating the portfolio with only big-size companies stocks could "beat the market", however, looking at the t-stat, we can conclude that none of the coefficients are significant, which is consistent with the theory.